# **Atividade 2 – Perguntas e Respostas usando Modelos do Hugging Face**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Dependências**

In [ ]:
!pip install -U --quiet transformers pypdf python-docx pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 142.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [ ]:
import pypdf
import docx
import pandas as pd
from transformers import pipeline
from transformers.utils import logging
import logging as py_logging
import warnings

### **Texto completo - Doenças Respiratórias Crônicas**

In [ ]:
doencas_cronicas = pypdf.PdfReader("/content/drive/MyDrive/TT/UFS/Eng de Computação/PLN/doencas_respiratorias_cronicas.pdf")
conteudo_doencas_cronicas = "\n".join([doencas_cronicas.pages[i].extract_text() for i in range(len(doencas_cronicas.pages))])

### **Texto completo - Dicionário de Dados**

In [ ]:
doc = docx.Document("/content/drive/MyDrive/TT/UFS/Eng de Computação/PLN/DICIONARIO_DE_DADOS.docx")
textos_completos = []
for para in doc.paragraphs:
    textos_completos.append(para.text)
for table in doc.tables:
    for row in table.rows:
        textos_completos.append(" | ".join(cell.text for cell in row.cells))
dict_dados = "\n".join(textos_completos)


### **Função principal**

In [ ]:
def responder_pergunta(qa_input: dict, model_name: str) -> dict:
    """
    Carrega um modelo de Question Answering e retorna a resposta para uma pergunta.
    Args:
        qa_input (dict): Dicionário com as chaves 'question' e 'context'.
        model_name (str): Nome do modelo de Question Answering a ser carregado.
    Returns:
        dict: Dicionário com as chaves 'answer', 'score' e 'start' e 'end'.
    """
    qa_model = pipeline("question-answering", model=model_name, tokenizer=model_name if model_name != "benjleite/ptt5-ptbr-qa" else "unicamp-dl/ptt5-base-portuguese-vocab")
    resultado = qa_model(qa_input)

    return resultado


In [ ]:
modelos = [
    "deepset/xlm-roberta-large-squad2",
    "pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    "benjleite/ptt5-ptbr-qa"
]

In [ ]:
warnings.filterwarnings('ignore')
logging.set_verbosity_error()
py_logging.getLogger("transformers").setLevel(py_logging.ERROR)

### **Doenças respiratórias**

In [ ]:
qa_doencas_respiratorias = [
    "Quais os tratamentos e atos de prevenção para rinite persistente grave?",
    "O que diferencia a hemoptise falsa da verdadeira?",
    "Quais os cuidados ambientais necessários para o controle das crises asmáticas?"
]

In [ ]:
for modelo in modelos:
    for pergunta in qa_doencas_respiratorias:
        qa_input = {
            "question": pergunta,
            "context": conteudo_doencas_cronicas
        }
        resposta = responder_pergunta(qa_input, modelo)
        print(f"Modelo: {modelo}\nPergunta: {pergunta}\nResposta: {resposta}\n\n")

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

Modelo: benjleite/ptt5-ptbr-qa
Pergunta: Quais os tratamentos e atos de prevenção para rinite persistente grave?
Resposta: {'score': 0.0004635002842405811, 'start': 65910, 'end': 65920, 'answer': ' teofilina'}


Modelo: benjleite/ptt5-ptbr-qa
Pergunta: O que diferencia a hemoptise falsa da verdadeira?
Resposta: {'score': 0.0009246234149031807, 'start': 52361, 'end': 52377, 'answer': ' broncodilatador'}


Modelo: benjleite/ptt5-ptbr-qa
Pergunta: Quais os cuidados ambientais necessários para o controle das crises asmáticas?
Resposta: {'score': 0.00031620135268894956, 'start': 44816, 'end': 44819, 'answer': ' de'}




### **Dicionário de Dados**

In [ ]:
qa_dic_dados = [
    "Quais são os possíveis tipos de serviço referenciado, de acordo com a tabela lfces019?",
    "Que tabelas se relacionam com a tb_carga_horaria_sus?",
    "Recebi um arquivo de planilha via email, que alegava ser uma amostra da tabela rl_estab_serv_class. Ao investigar o arquivo, notei que a planilha apresentava a coluna codservico com alguns registros nulos. Posso considerar esse arquivo como uma amostra verídica?"
]

In [ ]:
for modelo in modelos:
    for pergunta in qa_dic_dados:
        qa_input = {
            "question": pergunta,
            "context": dict_dados
        }
        resposta = responder_pergunta(qa_input, modelo)
        print(f"Modelo: {modelo}\nPergunta: {pergunta}\nResposta: {resposta}\n\n")

Modelo: deepset/xlm-roberta-large-squad2
Pergunta: Quais são os possíveis tipos de serviço referenciado, de acordo com a tabela lfces019?
Resposta: {'score': 0.12916484021116048, 'start': 586, 'end': 617, 'answer': '\nTABELA DE SERVIÇO REFERENCIADO'}


Modelo: deepset/xlm-roberta-large-squad2
Pergunta: Que tabelas se relacionam com a tb_carga_horaria_sus?
Resposta: {'score': 0.09939860133454204, 'start': 9429, 'end': 9473, 'answer': ' VÍNCULOS DO PROFISSIONAL NO ESTABELECIMENTO'}


Modelo: deepset/xlm-roberta-large-squad2
Pergunta: Recebi um arquivo de planilha via email, que alegava ser uma amostra da tabela rl_estab_serv_class. Ao investigar o arquivo, notei que a planilha apresentava a coluna codservico com alguns registros nulos. Posso considerar esse arquivo como uma amostra verídica?
Resposta: {'score': 0.09750110514079324, 'start': 20802, 'end': 20813, 'answer': ' VARCHAR(7)'}


Modelo: pierreguillou/bert-base-cased-squad-v1.1-portuguese
Pergunta: Quais são os possíveis tipos de

## **Implementando RAG**

In [ ]:
!pip install --quiet transformers torch sentencepiece faiss-cpu pypdf python-docx langchain sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 52.8 MB/s eta 0:00:00


In [ ]:
import torch
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks_doencas = text_splitter.split_text(conteudo_doencas_cronicas)
chunks_dict = text_splitter.split_text(dict_dados)

#Modelo de embedding
device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = SentenceTransformer("intfloat/multilingual-e5-large", device=device)

#Função para gerar embeddings
def get_embeddings(texts, model, batch_size=16):
    return model.encode(texts, batch_size=batch_size, convert_to_numpy=True, normalize_embeddings=True)

embeddings_doencas = get_embeddings(chunks_doencas, embedder)
embeddings_dict = get_embeddings(chunks_dict, embedder)


d1 = embeddings_doencas.shape[1]
index_doencas = faiss.IndexFlatL2(d1)
index_doencas.add(embeddings_doencas)

d2 = embeddings_dict.shape[1]
index_dict = faiss.IndexFlatL2(d2)
index_dict.add(embeddings_dict)


tokenizer_generator = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
model_generator = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct", device_map="auto", torch_dtype=torch.float16)
generator = pipeline("text-generation", model=model_generator, tokenizer=tokenizer_generator)


def qa_rag(pergunta: str, index, chunks_list, k: int = 5):
    embedding_pergunta = get_embeddings([pergunta], embedder)
    _, I = index.search(embedding_pergunta, k)

    documentos_relevantes = [chunks_list[i] for i in I[0]]
    contexto = "\n\n".join(documentos_relevantes)

    prompt = f"""
Você é um especialista.
Use APENAS as informações do contexto para responder em português claro, objetivo e resumido.
Se não encontrar no contexto, diga: "Não encontrei no contexto."

Contexto:
{contexto}

Pergunta: {pergunta}

Resposta:
    """

    outputs = generator(
        prompt,
        max_length=1024,
        temperature=0.2,
        top_p=0.85,
        num_return_sequences=1
    )
    resposta = outputs[0]["generated_text"].split("Resposta:")[-1].strip()
    return resposta

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for pergunta in qa_doencas_respiratorias:
    resposta = qa_rag(pergunta, index_doencas, chunks_doencas)
    print(f"Pergunta: {pergunta}\nResposta: {resposta}\n\n")

for pergunta in qa_dic_dados:
    resposta = qa_rag(pergunta, index_dict, chunks_dict)
    print(f"Pergunta: {pergunta}\nResposta: {resposta}\n\n")

Pergunta: Quais os tratamentos e atos de prevenção para rinite persistente grave?
Resposta: Para rinite persistente grave, o tratamento inclui:

1. **Medicamentos**: 
   - Corticoide inalatório nasal.
   - Anti-histamínico H1 oral (nas doses especificadas).

2. **Atos de Prevenção**:
   - Reduzir a exposição a fatores desencadeantes individuais.
   - Evitar exposição a ácaros ou alérgenos.
   - Evitar exposição a mofo.
   - Evitar tabagismo ativo e passivo.
   - Retirar animais domésticos se comprovada sensibilização.
   - Evitar odores fortes e exposição ocupacional.
   - Evitar locais de poluição atmosférica. **Não encontrei no contexto.** Não há informações específicas sobre atos de prevenção direcionados para rinite persistente grave no texto fornecido. O tratamento específico para essa condição é focado em medicamentos inalatórios e orais. Para rinite persistente grave, o tratamento deve ser prolongado por pelo menos 60 dias e reavaliado após uma semana. Se necessário, podem ser

